In [ ]:
import csv,time
from io import StringIO
from pyspark.sql import SparkSession

sc = SparkSession \
    .builder \
    .appName("Query 1 RDD") \
    .config("spark.executor.cores", "4") \
    .getOrCreate() \
    .sparkContext

def parse_csv_line(line):
    # Use StringIO to treat the line as a file-like object
    f = StringIO(line)
    # Use csv.reader to correctly parse the CSV line
    reader = csv.reader(f)
    return next(reader) 
def help1(data):
    try:
        age=int(data)
        if age<18 and age>0:
            return "child"
        if age<25:
            return "young adult"
        if age<65 :
            return "adult"
        if age>64:
            return "old"
        else:
            return "no individual victim"
    except:
        return "error"
    

start_time = time.time()
    
rdd1  = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv")\
.map(parse_csv_line)

rdd2= sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv")\
.map(parse_csv_line)

header1 = rdd1.first()
header2 = rdd2.first()

# Filter out the header
rdd1_data = rdd1.filter(lambda line: line != header1)
rdd2_data = rdd2.filter(lambda line: line != header2)

crime_data = rdd1_data.union(rdd2_data) \
.map(lambda data: (data[9], help1(data[11]))) \
.filter(lambda pair: pair[0].find("AGGRAVATED") != -1 ) \
.map(lambda data: (data[1], 1))  \
.reduceByKey(lambda a, b: a + b) \
.sortBy( lambda pair : pair[1], ascending=False )

print(crime_data.collect())

end_time = time.time()

print(f"Time taken: {end_time-start_time:.2f} seconds")

s3_path = "s3://groups-bucket-dblab-905418150721/group35/query1_RDD/"
crime_data.write.mode("overwrite").parquet(s3_path)








